### Updated on 14-May-2021
<br>
Sequential Version
<br>
In this file the implementation of N party protocol is shown. <br>
Here we have taken N = 5

In [1]:
from qiskit import *
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import matplotlib.pyplot as plotter
import numpy as np
from IPython.display import display, Math, Latex, Image
%matplotlib inline

Now here N people will try to follow a Quantum Conference Protocol with the help of an Untrusted (N+1)th Party 

In [2]:
num_bits = 40

key_ini = np.random.randint(2, size = num_bits)

p1_mess_ini = np.random.randint(2, size = num_bits)
p2_mess_ini = np.random.randint(2, size = num_bits)
p3_mess_ini = np.random.randint(2, size = num_bits)
p4_mess_ini = np.random.randint(2, size = num_bits)
p5_mess_ini = np.random.randint(2, size = num_bits)


qa_ini = np.random.choice(num_bits, size = num_bits, replace = False)
qb_ini = np.random.choice(num_bits, size = num_bits, replace = False)
qc_ini = np.random.choice(num_bits, size = num_bits, replace = False)
qd_ini = np.random.choice(num_bits, size = num_bits, replace = False)
qe_ini = np.random.choice(num_bits, size = num_bits, replace = False)

In [3]:
delta_m = 5

common_idx = np.sort(np.random.choice(num_bits, size = delta_m, replace = False))

In [4]:
def encode_message(key_bit, message_bit, qubit, circuit):
    if message_bit == 1:
        circuit.x(qubit)
    if key_bit == 1:
        circuit.h(qubit)
    return circuit

def run_circuit(qc, shot):
    backend = Aer.get_backend('qasm_simulator') 
    result = execute(qc, backend, shots = shot).result() 
    counts = result.get_counts()
    return counts

def cstr(counts):
    r = counts.items()
    for res, shots in r:
        result = str(res)
    return result

Now checking the Error in the channel 

In [5]:
result = []

for i in range(delta_m):
    
    a = QuantumRegister(1)
    b = QuantumRegister(1)
    c = QuantumRegister(1)
    d = QuantumRegister(1)
    e = QuantumRegister(1)
    ufp = ClassicalRegister(5)
    circuit = QuantumCircuit(a, b, c, d, e, ufp)
    
    idx = common_idx[i]
    
    #print(idx)
    
    a_mess = p1_mess_ini[idx]
    b_mess = p2_mess_ini[idx]
    c_mess = p3_mess_ini[idx]
    d_mess = p4_mess_ini[idx]
    e_mess = p5_mess_ini[idx]
    k = key_ini[idx]
    
    #print(a_mess, b_mess, c_mess, k)
    
    encode_message(k, a_mess, a[0], circuit)
    encode_message(k, b_mess, b[0], circuit)
    encode_message(k, c_mess, c[0], circuit)
    encode_message(k, d_mess, d[0], circuit)
    encode_message(k, e_mess, e[0], circuit)
    
    circuit.barrier()
    
    #Eaves Dropper could be introduced here so to check the exact security of the protocol with the threshold
    
    circuit.barrier()
    
    if(k == 1):
        circuit.h([a[0], b[0], c[0], d[0], e[0]])
    
    circuit.measure([a[0], b[0], c[0], d[0], e[0]], ufp)
    
    counts = run_circuit(circuit, 1)
    mess = cstr(counts)
    mess = mess[::-1]
    
    for j in range(len(mess)):
        result.append((int)(mess[j]))

result = np.array(result)

In [6]:
flag = 1

for i in range(delta_m):
    idx = common_idx[i]
    a = result[5*i]
    b = result[5*i+1]
    c = result[5*i+2]
    d = result[5*i+3]
    e = result[5*i+4]
    
    if a != p1_mess_ini[idx] or b != p1_mess_ini[idx] or c != p3_mess_ini[idx] or d != p4_mess_ini[idx] or e != p5_mess_ini[idx]:
        flag = 0

if flag == 0:
    print("Security Checking Failed !")
else:
    print("Security Checking Passed !")

Security Checking Failed !


In [7]:
key = []
p1_mess = []
p2_mess = []
p3_mess = []
p4_mess = []
p5_mess = []

for i in range(num_bits):
    if i not in common_idx:
        key.append(key_ini[i])
        p1_mess.append(p1_mess_ini[i])
        p2_mess.append(p2_mess_ini[i])
        p3_mess.append(p3_mess_ini[i])
        p4_mess.append(p4_mess_ini[i])
        p5_mess.append(p5_mess_ini[i])
        
key = np.array(key)
p1_mess = np.array(p1_mess)
p2_mess = np.array(p2_mess)
p3_mess = np.array(p3_mess)
p4_mess = np.array(p4_mess)
p5_mess = np.array(p5_mess)
new_num_bits = len(p5_mess)

In [8]:
final_result = []

for i in range(new_num_bits):
    
    a = QuantumRegister(1)
    b = QuantumRegister(1)
    c = QuantumRegister(1)
    d = QuantumRegister(1)
    e = QuantumRegister(1)
    ufp = ClassicalRegister(5)
    circuit = QuantumCircuit(a, b, c, d, e, ufp)
    
    a_mess = p1_mess[i]
    b_mess = p2_mess[i]
    c_mess = p3_mess[i]
    d_mess = p4_mess[i]
    e_mess = p5_mess[i]
    k = key[i]
    #print(a_mess, b_mess, c_mess, k)
    
    encode_message(k, a_mess, a[0], circuit)
    encode_message(k, b_mess, b[0], circuit)
    encode_message(k, c_mess, c[0], circuit)
    encode_message(k, d_mess, d[0], circuit)
    encode_message(k, e_mess, e[0], circuit)
    
    circuit.barrier()
    
    circuit.cx(a[0], b[0])
    circuit.cx(a[0], c[0])
    circuit.cx(a[0], d[0])
    circuit.cx(a[0], e[0])
    circuit.h(a[0])
    
    circuit.barrier()
    
    circuit.measure([a[0], b[0], c[0], d[0], e[0]], ufp)
    
    counts = run_circuit(circuit, 1)
    mess = cstr(counts)
    mess = mess[::-1]
    
    #print(mess)
    for j in range(len(mess)):
        final_result.append((int)(mess[j]))

final_result = np.array(final_result)

#### Decryption of Message 

In [9]:
dec_12 = np.empty(shape = new_num_bits, dtype = int)  # This dec_ij signifies the message of j decrypted by i
dec_13 = np.empty(shape = new_num_bits, dtype = int)
dec_14 = np.empty(shape = new_num_bits, dtype = int)
dec_15 = np.empty(shape = new_num_bits, dtype = int)

dec_21 = np.empty(shape = new_num_bits, dtype = int)
dec_23 = np.empty(shape = new_num_bits, dtype = int)
dec_24 = np.empty(shape = new_num_bits, dtype = int)
dec_25 = np.empty(shape = new_num_bits, dtype = int)

dec_32 = np.empty(shape = new_num_bits, dtype = int)
dec_31 = np.empty(shape = new_num_bits, dtype = int)
dec_34 = np.empty(shape = new_num_bits, dtype = int)
dec_35 = np.empty(shape = new_num_bits, dtype = int)

dec_42 = np.empty(shape = new_num_bits, dtype = int)
dec_43 = np.empty(shape = new_num_bits, dtype = int)
dec_41 = np.empty(shape = new_num_bits, dtype = int)
dec_45 = np.empty(shape = new_num_bits, dtype = int)

dec_52 = np.empty(shape = new_num_bits, dtype = int)
dec_53 = np.empty(shape = new_num_bits, dtype = int)
dec_54 = np.empty(shape = new_num_bits, dtype = int)
dec_51 = np.empty(shape = new_num_bits, dtype = int)

one_idx = []
p1_xor = []
p2_xor = []
p3_xor = []
p4_xor = []
p5_xor = []

In [10]:
for i in range(new_num_bits):
    
    if key[i] == 0:
        
        p1 = (int)(final_result[5*i])
        p2 = (int)(final_result[5*i+1])
        p3 = (int)(final_result[5*i+2])
        p4 = (int)(final_result[5*i+3])
        p5 = (int)(final_result[5*i+4])
        
        if p1_mess[i] == 0:
            dec_12[i] = p2
            dec_13[i] = p3
            dec_14[i] = p4
            dec_15[i] = p5
        else:
            dec_12[i] = 1 - p2
            dec_13[i] = 1 - p3
            dec_14[i] = 1 - p4
            dec_15[i] = 1 - p5
            
        if p2_mess[i] == p2:
            dec_21[i] = 0
            dec_23[i] = p3
            dec_24[i] = p4
            dec_25[i] = p5
        else:
            dec_21[i] = 1
            dec_23[i] = 1 - p3
            dec_24[i] = 1 - p4
            dec_25[i] = 1 - p5
            
        if p3_mess[i] == p3:
            dec_31[i] = 0
            dec_32[i] = p2
            dec_34[i] = p4
            dec_35[i] = p5
        else:
            dec_31[i] = 1
            dec_32[i] = 1 - p2
            dec_34[i] = 1 - p4
            dec_35[i] = 1 - p5
            
        if p4_mess[i] == p4:
            dec_41[i] = 0
            dec_43[i] = p3
            dec_42[i] = p2
            dec_45[i] = p5
        else:
            dec_41[i] = 1
            dec_43[i] = 1 - p3
            dec_42[i] = 1 - p2
            dec_45[i] = 1 - p5
            
        if p5_mess[i] == p5:
            dec_51[i] = 0
            dec_53[i] = p3
            dec_54[i] = p4
            dec_52[i] = p2
        else:
            dec_51[i] = 1
            dec_53[i] = 1 - p3
            dec_54[i] = 1 - p4
            dec_52[i] = 1 - p2
            
    
    else:
        
        one_idx.append(i)
        chk = (int)(final_result[5*i])
        
        if p1_mess[i] == chk:
            p1_xor.append(0)
        else:
            p1_xor.append(1)
        
        if p2_mess[i] == chk:
            p2_xor.append(0)
        else:
            p2_xor.append(1)
        
        if p3_mess[i] == chk:
            p3_xor.append(0)
        else:
            p3_xor.append(1)
        
        if p4_mess[i] == chk:
            p4_xor.append(0)
        else:
            p4_xor.append(1)
        
        if p5_mess[i] == chk:
            p5_xor.append(0)
        else:
            p5_xor.append(1)

In [11]:
num_decoy = 5

decoy_1_base = np.random.randint(2, size = num_decoy)
decoy_1_bit = np.random.randint(2, size = num_decoy)

decoy_2_base = np.random.randint(2, size = num_decoy)
decoy_2_bit = np.random.randint(2, size = num_decoy)

decoy_3_base = np.random.randint(2, size = num_decoy)
decoy_3_bit = np.random.randint(2, size = num_decoy)

decoy_4_base = np.random.randint(2, size = num_decoy)
decoy_4_bit = np.random.randint(2, size = num_decoy)

decoy_5_base = np.random.randint(2, size = num_decoy)
decoy_5_bit = np.random.randint(2, size = num_decoy)

flag = 1

Now in the next five cells security has been checked for each circular channel 